## Frontend Development

The whole project folder of a APP should look like this:

In [ ]:
backend/        # your FastAPI app
  main.py
  requirements.txt
frontend/       # your Next.js app
  package.json
  ...

### Next js

We create a test project named app_test with folder structure

In [ ]:
backend/        
  main.py
  requirements.txt
frontend/       

With such folder structure, we can execute below command to create a ready-to-run Next.js project named frontend with TypeScript, ESLint, and Tailwind preconfigured.

In [ ]:
npx create-next-app@latest frontend --ts --eslint --tailwind #run the command at test_app root dir, brew install node first

Break down the command:
- npx: a package manager for javascript (allow us to run the package without installing it globally).
- create-next-app@latest: the official Next.js project generator (using the newest version).
- frontend: the folder/name for frontend.
- ts: sets up TypeScript (tsconfig.json, next-env.d.ts, .tsx pages/components).
- eslint: adds ESLint with Next.js rules (includes next/core-web-vitals) and a .eslintrc. ESLint is a static analysis (linting) tool for JavaScript/TypeScript. It scans your code without running it and flags problems—bugs, anti-patterns, style issues, and security foot-guns—based on a set of rules.
- tailwind: installs and wires Tailwind CSS (tailwind.config.js, postcss.config.js, adds @tailwind directives to globals.css)


We get:
- A Next.js app (App Router by default) with sensible defaults.
- Dependencies installed (next, react, react-dom, tailwindcss, postcss, autoprefixer, ESLint plugins).
- Git initialized (unless disabled), .gitignore, README.md, and npm scripts in package.json.

In [ ]:
.
├── backend
│   ├── README.md
│   └── main.py
└── frontend
    ├── README.md
    ├── eslint.config.mjs
    ├── next-env.d.ts
    ├── next.config.ts
    ├── node_modules
    ├── package-lock.json
    ├── package.json
    ├── postcss.config.mjs
    ├── public
    ├── src
    └── tsconfig.json

To run the app, we use following command:

In [ ]:
cd frontend
npm run dev

This starts the devlopment server (for our frontend app) on http://localhost:3000, we can open it using our browser.

Next, we need to allow frontend to be able to to call from our backend; we achieve this through adding below code to main.py of the backend.

In [ ]:
# fastapi main.py
from fastapi.middleware.cors import CORSMiddleware

app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:3000"],  # your Next.js dev URL
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

We add an environment variable to link the ports. For that we create: frontend/.env.local and write:

In [ ]:
NEXT_PUBLIC_API_URL=http://127.0.0.1:8000

After adding above file, we can see that the next.js frontend detect change in .env.local and automatically reload.

The frontend can now access backend, now we only need to modify the page content of the frontend. Then, in frontend/src/app/page.tsx, we delete all content and we paste:

In [ ]:
// app/page.tsx
"use client";
import { useState } from "react";

type FftResponse = {
  k: number; n: number; L: number;
  bin_index: number;
  ideal_bin_float: number;
  bin_offset_from_integer: number;
  Y_m_real: number; Y_m_imag: number; Y_m_abs: number;
  Y_m_norm_real: number; Y_m_norm_imag: number; Y_m_norm_abs: number;
  note: string;
};

export default function Page() {
  const [n, setN] = useState(4096);
  const [L, setL] = useState(2 * Math.PI);
  const [k, setK] = useState(14);
  const [data, setData] = useState<FftResponse | null>(null);
  const [loading, setLoading] = useState(false);
  const [err, setErr] = useState<string | null>(null);

  const run = async () => {
    try {
      setLoading(true); setErr(null);
      const url = new URL(`${process.env.NEXT_PUBLIC_API_URL}/fft`);
      url.search = new URLSearchParams({
        n: String(n),
        L: String(L),
        k: String(k),
      }).toString();

      const res = await fetch(url.toString());
      if (!res.ok) throw new Error(await res.text());
      const j = (await res.json()) as FftResponse;
      setData(j);
    } catch (e: any) {
      setErr(e.message ?? "Request failed");
    } finally {
      setLoading(false);
    }
  };

  return (
    <main className="p-6 space-y-4">
      <h1 className="text-2xl font-bold">Signal Lab UI</h1>

      <div className="grid grid-cols-3 gap-3 max-w-xl">
        <label className="flex flex-col">
          <span>N</span>
          <input className="border rounded p-2" type="number" value={n}
                 onChange={e => setN(parseInt(e.target.value || "0"))}/>
        </label>
        <label className="flex flex-col">
          <span>L</span>
          <input className="border rounded p-2" type="number" step="any" value={L}
                 onChange={e => setL(parseFloat(e.target.value || "0"))}/>
        </label>
        <label className="flex flex-col">
          <span>k</span>
          <input className="border rounded p-2" type="number" value={k}
                 onChange={e => setK(parseInt(e.target.value || "0"))}/>
        </label>
      </div>

      <button
        onClick={run}
        disabled={loading}
        className="px-4 py-2 rounded-xl shadow bg-black text-white disabled:opacity-50"
      >
        {loading ? "Computing…" : "Compute FFT bin"}
      </button>

      {err && <p className="text-red-600">{err}</p>}

      {data && (
        <pre className="p-4 rounded-xl bg-gray-100 overflow-auto">
          {JSON.stringify(data, null, 2)}
        </pre>
      )}
    </main>
  );
}

We can now see our frontend app by again open the url http://localhost:3000/ in our browser.

It is useful to use LLMs for vibe coding the user-interface of the frontend (send the backend main.py code and ask for frontend page.tsx file).